# Reforma

## Preâmbulo

In [1]:
import numpy as np
import pandas as pd
import psycopg2
import pandas.io.sql as sqlio
import time

# Track execution time                                                          
start_time = time.time()

### Parâmetros

In [2]:
# Simulation Parameters
#TB_REFORMA = 'FATO_REFORMA_SAMPLE'
TB_REFORMA = 'FATO_REFORMA'
ANO_INICIO = 1995
ANO_FIM = 2016
DADOS_POPULACAO = '../dataset/PROJECOES_2013_POPULACAO.xls'
DADOS_FAZENDA = '../dataset/dados_fazenda.xlsx'
OUTPUT_FILE = '../outputs/reforma_analisys.xls'

writer = pd.ExcelWriter(OUTPUT_FILE)

# Conection parameters
HOST='172.17.0.1'
PORT='5432'
DBNAME='prevdb'
USER='prevdb_user'
PASS='pr3v'

### Biblioteca

In [3]:
def ds_query(sql_query):
    """
        Query Dataset

    Parâmetros
    ----------
        sql : string
            SQL query to be performed against the dataset
        
    Retorno
    -------
        Pandas Dataframe
    """
    # Connect to an existing database
    try:
        conn = psycopg2.connect("host='{}' port={} dbname='{}'user={} password={}"
                .format(HOST, PORT, DBNAME, USER, PASS))
        df = sqlio.read_sql_query(sql, conn)
        # Close communication with the database                                     
        conn.close()
        return df
    except:
        print("Unable to connect to the database")
        return

## Simulação

### População

In [4]:
### População                                                                      
POPH = pd.read_excel(DADOS_POPULACAO,                                                
                     sheet_name='BRH',                                        
                     index_col='IDADE',                                           
                     nrows=91,                                                     
                     dtype=int)  
POPM = pd.read_excel(DADOS_POPULACAO,                                                
                     sheet_name='BRM',                                        
                     index_col='IDADE',                                           
                     nrows=91,                                                     
                     dtype=int) 

### Cálculo da idade média de aposentadoria com a PEC 

#### PEC

In [5]:
sql = """
SELECT SEXO, PEC6_IDADE_DIB, ESPECIE, CLIENTELA
FROM {table_name} 
WHERE PEC6_ANO_DIB = {ano}
""".format(table_name=TB_REFORMA,
           ano=ANO_FIM)
df = ds_query(sql)
df.head()

,sexo,pec6_idade_dib,especie,clientela
0,1,65.0,42,1
1,1,65.0,42,1
2,1,65.0,42,1
3,3,62.0,42,1
4,1,60.0,46,1


In [6]:
df_avg = df.pivot_table(index='sexo', columns=['especie','clientela'], 
                    values='pec6_idade_dib', aggfunc=np.mean)
df_avg.fillna(value=0, inplace=True, downcast='infer')
df_avg.to_excel(writer, '2016_pec6_idade_media')
df_avg

especie           32                    41                    42             \
clientela          1          2          1          2          1          2   
sexo                                                                          
1          52.835387  51.981002  68.002831  63.050255  65.025879  60.153779   
3          53.187058  49.769820  65.170822  61.842023  62.048980  60.012658   

especie           46             57         92             
clientela          1   2          1          1          2  
sexo                                                       
1          59.750164  60  60.689904  50.708412  52.200000  
3          59.911814   0  60.763121  50.898582  51.511364

In [7]:
# MEDIA GERAL
sql = """
SELECT SEXO, AVG(PEC6_IDADE_DIB) AVG_PEC
FROM {table_name} 
WHERE PEC6_ANO_DIB = {ano}
GROUP BY SEXO
""".format(table_name=TB_REFORMA,
           ano=ANO_FIM)
df = ds_query(sql)
df.head()

,sexo,avg_pec
0,1,63.327610
1,3,62.313563


#### RGPS

In [8]:
sql = """
SELECT SEXO, IDADE_DIB, ESPECIE, CLIENTELA
FROM {table_name} 
WHERE ANO_DIB = {ano}
""".format(table_name=TB_REFORMA,
           ano=ANO_FIM)
df = ds_query(sql)
df.head()

,sexo,idade_dib,especie,clientela
0,1,49,42,1
1,1,60,42,1
2,3,46,42,1
3,1,65,41,1
4,1,54,42,1


In [9]:
df_avg = df.pivot_table(index='sexo', columns=['especie', 'clientela'], 
                    values='idade_dib', aggfunc=np.mean)
df_avg.fillna(value=0, inplace=True, downcast='infer')
df_avg.to_excel(writer, '2016_rgps_idade_media')
df_avg

especie           32                    41                    42             \
clientela          1          2          1          2          1          2   
sexo                                                                          
1          52.835387  51.981002  65.561154  60.510512  56.331422  54.561753   
3          53.187058  49.769820  61.421269  55.950077  53.281560  49.454545   

especie           46               57         92             
clientela          1     2          1          1          2  
sexo                                                         
1          48.569503  50.8  55.901024  50.708412  52.200000  
3          49.328877   0.0  52.461551  50.898582  51.511364

In [10]:
# MEDIA GERAL
sql = """
SELECT SEXO, AVG(IDADE_DIB) AVG_PEC
FROM {table_name} 
WHERE ANO_DIB = {ano}
GROUP BY SEXO
""".format(table_name=TB_REFORMA,
           ano=ANO_FIM)
df = ds_query(sql)
df.head()

,sexo,avg_pec
0,1,58.800691
1,3,57.069440


### Probabilidade de se aposentar

**Nasceu entre 1935 e 1940 e se aposentou entre 2000 e 2016**

[12:20, 27/5/2019] Denise Ufrj: 1. Qual a probabilidade de uma pessoa se aposentar, homem e mulher (por idade, professor, rural, especial em 2016), com as regras vigentes hoje, e qual  seria a probabilidade de se aposentar em 2016 se as regras da PEC 6 estivessem em vigor?
HOMENS:
Pt = [ Ct_2000(65) + Ct_2001(65..66) + Ct_2002(65..67) + Ct_2003(65..68) + Ct_2004(65..69) + Ct_2005(65..70) + Ct_2006(66..71) + … Ct_2005+i(65+i..70+i) … + Ct_2016(76..81) ] / 
2320805 
pessoas de 2000 a 2005, que completaram 65 anos: quantas delas de aposetaram (proporção) – até 2016
Mulheres:
Pt = [ Ct_2000(60) + Ct_2001(60..61) + Ct_2002(60..62) + Ct_2003(60..63) + Ct_2004(60..64) + Ct_2005(60..65) + Ct_2006(61..66) + … Ct_2005+i(60+i..65+i) … + Ct_2016(71..76) ] / 
3289742


In [11]:
#poph65_2000a2005
poph65_2000a2005 = 0
for i in range(2000,2006):
    poph65_2000a2005 += POPH.loc[65][i]
poph65_2000a2005

2320805

#### PEC

In [12]:
# Nasceu entre 1935 e 1940 e se aposentou entre 2000 e 2016
sql = """
SELECT
	ESPECIE
	,SEXO
	,CLIENTELA
	,COUNT(ANO_NASC) AS TOTAL
FROM {table_name}
WHERE 
	PEC6_ANO_DIB <= 2016
	AND ANO_NASC >= 1935 AND ANO_NASC <= 1940
GROUP BY
	ESPECIE
	,SEXO
	,CLIENTELA
""".format(table_name=TB_REFORMA,
           ano=ANO_FIM)
df = ds_query(sql)
df['pec_prob_aposentar'] = df['total'].apply(lambda x: x/poph65_2000a2005)
df.to_excel(writer, 'pec_prob_aposentar')
df

,especie,sexo,clientela,total,pec_prob_aposentar
0,32,1,1,142812,6.153554e-02
1,32,1,2,24436,1.052911e-02
2,32,3,1,137471,5.923419e-02
3,32,3,2,5426,2.337982e-03
4,41,1,1,355420,1.531451e-01
5,41,1,2,602827,2.597491e-01
6,41,3,1,434449,1.871975e-01
7,41,3,2,343340,1.479400e-01
8,42,1,1,212203,9.143508e-02
9,42,1,2,1714,7.385368e-04


#### RGPS

In [13]:
# Nasceu entre 1935 e 1940 e se aposentou entre 2000 e 2016
sql = """
SELECT
	ESPECIE
	,SEXO
	,CLIENTELA
	,COUNT(ANO_NASC) AS TOTAL
FROM {table_name}
WHERE 
	ANO_DIB <= 2016
	AND ANO_NASC >= 1935 AND ANO_NASC <= 1940
GROUP BY
	ESPECIE
	,SEXO
	,CLIENTELA
""".format(table_name=TB_REFORMA,
           ano=ANO_FIM)
df = ds_query(sql)
df['rgps_prob_aposentar'] = df['total'].apply(lambda x: x/poph65_2000a2005)
df.to_excel(writer, 'rgps_prob_aposentar')
df

,especie,sexo,clientela,total,rgps_prob_aposentar
0,32,1,1,142812,6.153554e-02
1,32,1,2,24436,1.052911e-02
2,32,3,1,137471,5.923419e-02
3,32,3,2,5426,2.337982e-03
4,41,1,1,369987,1.594218e-01
5,41,1,2,648881,2.795931e-01
6,41,3,1,462031,1.990822e-01
7,41,3,2,413771,1.782877e-01
8,42,1,1,212203,9.143508e-02
9,42,1,2,1714,7.385368e-04


## Proporção dos que não cumpriram a regra (GAP > 0)

Proporção de quantos não cumpriram a regra da PEC por quantos tinham se aposentado em 2016 (para os dois sexos e para todos os tipos)

[12:25, 27/5/2019] Denise Ufrj: 3. Juntar, mais a frente, com os dados da PNAD Continua (taxa de desemprego, taxa de participação, formalização) de 2019 para calibrar ainda mais as probabilidade.

In [14]:
sql = """
SELECT 
	ESPECIE
	,SEXO
	,CLIENTELA
	,COUNT(PEC6_GAP) AS TOTAL
FROM FATO_REFORMA
WHERE ANO_DIB = 2016
	AND PEC6_GAP > 0
GROUP BY
	ESPECIE
	,CLIENTELA
	,SEXO
ORDER BY 
	ESPECIE ASC
	,SEXO ASC
	,CLIENTELA ASC
""".format(table_name=TB_REFORMA,
           ano=ANO_FIM)
df = ds_query(sql)
df.to_excel(writer, 'rgps_gap_qtd')
df

,especie,sexo,clientela,total
0,41,1,1,61215
1,41,1,2,55581
2,41,3,1,174136
3,41,3,2,106574
4,42,1,1,156381
5,42,1,2,956
6,42,3,1,102541
7,42,3,2,44
8,46,1,1,3546
9,46,1,2,5


In [15]:
# TODOS RGPS 2016
sql = """
SELECT 
	ESPECIE
	,SEXO
	,CLIENTELA
	,COUNT(PEC6_GAP) AS TOTAL
FROM FATO_REFORMA
WHERE ANO_DIB = 2016
GROUP BY
	ESPECIE
	,CLIENTELA
	,SEXO
ORDER BY 
	ESPECIE ASC
	,SEXO ASC
	,CLIENTELA ASC
""".format(table_name=TB_REFORMA,
           ano=ANO_FIM)
df = ds_query(sql)
df.to_excel(writer, 'rgps_2016_qtd')
df

,especie,sexo,clientela,total
0,32,1,1,62990
1,32,1,2,7422
2,32,3,1,38325
3,32,3,2,4566
4,41,1,1,108145
5,41,1,2,97741
6,41,3,1,176443
7,41,3,2,108247
8,42,1,1,157633
9,42,1,2,1004


## Proporção do Tempo de Contribuição (QTD_TC/TOTAL)

In [16]:
# ESPECIE 42
sql = """
SELECT 
	CLIENTELA
	,SEXO
	,COUNT(PEC6_GAP) AS TOTAL
FROM {table_name}
WHERE ESPECIE = 42
GROUP BY
	CLIENTELA
	,SEXO
ORDER BY 
	SEXO ASC
	,CLIENTELA ASC
""".format(table_name=TB_REFORMA,
           ano=ANO_FIM)
df = ds_query(sql)
df.to_excel(writer, 'rgps_qtd_42')
df

,clientela,sexo,total
0,1,1,3464564
1,2,1,22361
2,1,3,1550958
3,2,3,1439


In [17]:
# TODOS
sql = """
SELECT 
	CLIENTELA
	,SEXO
	,COUNT(PEC6_GAP) AS TOTAL
FROM {table_name}
GROUP BY
	CLIENTELA
	,SEXO
ORDER BY 
	SEXO ASC
	,CLIENTELA ASC
""".format(table_name=TB_REFORMA,
           ano=ANO_FIM)
df = ds_query(sql)
df.to_excel(writer, 'rgps_qtd_todos')
df

,clientela,sexo,total
0,1,1,7237010
1,2,1,3042237
2,1,3,5472954
3,2,3,3785388


## Tempo médio de Contribuição

In [18]:
sql = """
SELECT 
	ESPECIE
	,SEXO
	,CLIENTELA
	,AVG(TEMPO_CONTRIB) AS AVG_CONTRIB_RGPS
	,AVG(TEMPO_CONTRIB + PEC6_GAP) AS AVG_CONTRIB_PEC6
	,AVG(PEC6_GAP) AS AVG_GAP
FROM {table_name}
GROUP BY
	ESPECIE
	,CLIENTELA
	,SEXO
ORDER BY 
	ESPECIE ASC
	,SEXO ASC
	,CLIENTELA ASC
""".format(table_name=TB_REFORMA,
           ano=ANO_FIM)
df = ds_query(sql)
df.to_excel(writer, 'avg_contrib_todos')
df

,especie,sexo,clientela,avg_contrib_rgps,avg_contrib_pec6,avg_gap
0,32,1,1,9.822513,9.822513,0.000000
1,32,1,2,5.778481,5.778481,0.000000
2,32,3,1,6.248774,6.248774,0.000000
3,32,3,2,4.296615,4.296615,0.000000
4,41,1,1,20.055217,22.997227,2.942009
5,41,1,2,15.806692,22.728967,6.922275
6,41,3,1,17.010762,21.868111,4.857349
7,41,3,2,13.639937,23.402990,9.763053
8,42,1,1,34.521325,46.016048,11.494723
9,42,1,2,34.964134,40.757569,5.793435


## Cleanup

In [19]:
# Write output file
writer.save()

# Print out elapsed time                                                        
elapsed_time = (time.time() - start_time) / 60                                  
print("\nExecution time: {0:0.4f} minutes.".format(elapsed_time))


Execution time: 0.7910 minutes.
